The purpose of this notebook is to **explore, understand, and preprocess** the dataset — preparing a clean version suitable for **machine learning modeling** in the next phase.

In [1]:
import numpy as np
import pandas as pd

In [2]:
dff = pd.read_csv("../data/raw/BRFSS-2015.csv")

#### Important Risk Factors
Selecting the following  **important risk factors** for diabetes and other chronic illnesses like heart disease (not in strict order of importance):

*   blood pressure (high)
*   cholesterol (high)
*   smoking
*   diabetes
*   age
*   sex
*   race
*   diet
*   exercise
*   alcohol consumption
*   BMI
*   Household Income
*   Marital Status
*   Sleep
*   Time since last checkup
*   Education
*   Health care coverage
*   Mental Health

#### I consulted the codebook for what each column / question is to understand which column to select and how to clean the data here  [https://www.cdc.gov/brfss/annual_data/2015/pdf/codebook15_llcp.pdf](https://www.cdc.gov/brfss/annual_data/2015/pdf/codebook15_llcp.pdf).
 Then I selected those relevant to diabetes 

In [3]:
# columns in dataset  that are important risk factors of diabetes
diabetes_factors = ["DIABETE3","_RFHYPE5","TOLDHI2","_CHOLCHK","_BMI5","SMOKE100",
                    "CVDSTRK3", "_MICHD","_TOTINDA","_FRTLT1", "_VEGLT1", "_RFDRHV5",
                    "HLTHPLN1","MEDCOST","GENHLTH","MENTHLTH","PHYSHLTH","DIFFWALK","SEX",
                    "_AGEG5YR","EDUCA", "INCOME2"]

In [4]:
# working on a copy of selected columns
df = dff.loc[:, diabetes_factors].copy()
print(f"Shape :{df.shape}")
df.head()

Shape :(441456, 22)


,DIABETE3,_RFHYPE5,TOLDHI2,_CHOLCHK,_BMI5,SMOKE100,CVDSTRK3,_MICHD,_TOTINDA,_FRTLT1,...,HLTHPLN1,MEDCOST,GENHLTH,MENTHLTH,PHYSHLTH,DIFFWALK,SEX,_AGEG5YR,EDUCA,INCOME2
0,3.0,2.0,1.0,1.0,4018.0,1.0,2.0,2.0,2.0,2.0,...,1.0,2.0,5.0,18.0,15.0,1.0,2.0,9.0,4.0,3.0
1,3.0,1.0,2.0,2.0,2509.0,1.0,2.0,2.0,1.0,2.0,...,2.0,1.0,3.0,88.0,88.0,2.0,2.0,7.0,6.0,1.0
2,3.0,1.0,1.0,1.0,2204.0,NaN,1.0,NaN,9.0,9.0,...,1.0,2.0,4.0,88.0,15.0,NaN,2.0,11.0,4.0,99.0
3,3.0,2.0,1.0,1.0,2819.0,2.0,2.0,2.0,2.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,2.0,9.0,4.0,8.0
4,3.0,1.0,2.0,1.0,2437.0,2.0,2.0,2.0,2.0,9.0,...,1.0,2.0,5.0,88.0,20.0,2.0,2.0,9.0,5.0,77.0


In [5]:
# defined codes in the dataset that represent missing values from the codebook
missing_codes = {
    "DIABETE3": [7,9],
    "_RFHYPE5": [9],
    "TOLDHI2": [7,9],
    "_CHOLCHK": [9],
    "SMOKE100": [7,9],
    "CVDSTRK3": [7,9],
    "_TOTINDA": [9],
    "_FRTLT1": [9],
    "_VEGLT1": [9],
    "_RFDRHV5": [9],
    "HLTHPLN1": [7,9],
    "MEDCOST": [7,9],
    "GENHLTH": [7,9],
    "MENTHLTH": [77,99],
    "PHYSHLTH": [77,99],
    "DIFFWALK": [7,9],
    "_AGEG5YR": [14],
    "EDUCA": [9],
    "INCOME2": [77,99]
}

In [6]:
for col, codes in missing_codes.items():
    df[col] = df[col].replace({c:np.nan for c in codes})

In [7]:
# total number of null values in each column
df.isnull().sum()

DIABETE3      798
_RFHYPE5     1367
TOLDHI2     62715
_CHOLCHK    15337
_BMI5       36398
SMOKE100    17549
CVDSTRK3     1290
_MICHD       3942
_TOTINDA    37992
_FRTLT1     43711
_VEGLT1     51117
_RFDRHV5    25733
HLTHPLN1     1840
MEDCOST      1194
GENHLTH      1247
MENTHLTH     7408
PHYSHLTH     9743
DIFFWALK    14594
SEX             0
_AGEG5YR     5336
EDUCA        1811
INCOME2     79505
dtype: int64

In [8]:
# checking for duplicates 
df.duplicated().value_counts()

False    433732
True       7724
Name: count, dtype: int64

In [9]:
# removing duplicates to prevent data leakage and ensure data integrity
df.drop_duplicates(inplace=True)

In [10]:
# converting _BMI5 to float and scaling it down by 100 to get actual BMI values
df["_BMI5"] = (df["_BMI5"].astype('float').div(100)).round(1)

In [11]:
# dropping rows with any coded-missing values
before = len(df)
df.dropna(axis="index", how="any", inplace=True)
after = len(df)
print(f"Rows before coded-missing removal: {before}, after: {after}, dropped: {before-after}")

Rows before coded-missing removal: 433732, after: 249344, dropped: 184388


In [12]:
# renaming columns for better readability
df.rename(mapper={"DIABETE3":"Diabetes","_RFHYPE5":"HighBp","TOLDHI2":"HighCholesterol",
                  "_CHOLCHK":"CheckedCholesterol","_BMI5":"BMI","SMOKE100":"Smoke100",
                  "CVDSTRK3":"Stroke","_MICHD":"HeartDisease","_TOTINDA":"PhysicalActivity",
                  "_FRTLT1":"Fruits","_VEGLT1":"Vegetables","_RFDRHV5":"HeavyDrinking",
                  "HLTHPLN1":"HealthCareAccess","MEDCOST":"MedicalCost","GENHLTH":"GeneralHealth",
                  "MENTHLTH":"MentalHealth","PHYSHLTH":"PhysicalHealth","DIFFWALK":"DifficultyWalking",
                  "SEX":"Sex","_AGEG5YR":"Age","EDUCA":"Education","INCOME2":"Income"},axis=1,inplace=True)
df.head()

,Diabetes,HighBp,HighCholesterol,CheckedCholesterol,BMI,Smoke100,Stroke,HeartDisease,PhysicalActivity,Fruits,...,HealthCareAccess,MedicalCost,GeneralHealth,MentalHealth,PhysicalHealth,DifficultyWalking,Sex,Age,Education,Income
0,3.0,2.0,1.0,1.0,40.2,1.0,2.0,2.0,2.0,2.0,...,1.0,2.0,5.0,18.0,15.0,1.0,2.0,9.0,4.0,3.0
1,3.0,1.0,2.0,2.0,25.1,1.0,2.0,2.0,1.0,2.0,...,2.0,1.0,3.0,88.0,88.0,2.0,2.0,7.0,6.0,1.0
3,3.0,2.0,1.0,1.0,28.2,2.0,2.0,2.0,2.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,2.0,9.0,4.0,8.0
5,3.0,2.0,2.0,1.0,26.5,2.0,2.0,2.0,1.0,1.0,...,1.0,2.0,2.0,88.0,88.0,2.0,2.0,11.0,3.0,6.0
6,3.0,2.0,1.0,1.0,23.9,2.0,2.0,2.0,1.0,1.0,...,1.0,2.0,2.0,3.0,88.0,2.0,2.0,11.0,5.0,4.0


In [13]:
# mapping categorical encodings consistently across columns
# Reversing  the  ordinal scale of GeneralHealth for consistency:
df["GeneralHealth"] = df["GeneralHealth"].map({1:5,2:4,3:3,4:2,5:1})

In [14]:
#  Replacing 88 (not applicable) with 0 in MentalHealth and PhysicalHealth columns
df["MentalHealth"] = df["MentalHealth"].replace({88:0})
df["PhysicalHealth"] = df["PhysicalHealth"].replace({88:0})
#  Binary encoding for categorical variables
df["DifficultyWalking"] = df["DifficultyWalking"].replace({1:1,2:0})
df["Sex"] = df["Sex"].replace({1:1,2:0})
df["HeavyDrinking"] = df["HeavyDrinking"].replace({1:0,2:1})
df["HealthCareAccess"] = df["HealthCareAccess"].replace({1:1,2:0})
df["MedicalCost"] = df["MedicalCost"].replace({1:1,2:0})
df["PhysicalActivity"] = df["PhysicalActivity"].replace({1:1,2:0})
df["Fruits"] = df["Fruits"].replace({1:1,2:0})
df["Vegetables"] = df["Vegetables"].replace({1:1,2:0})
df["Smoke100"] = df["Smoke100"].replace({1:1,2:0})
df["Stroke"] = df["Stroke"].replace({1:1,2:0})
df["HeartDisease"] = df["HeartDisease"].replace({1:1,2:0})
df["HighBp"] = df["HighBp"].replace({1:0,2:1})
df["HighCholesterol"] = df["HighCholesterol"].replace({2:0,1:1})
df["CheckedCholesterol"] = df["CheckedCholesterol"].replace({1:1,2:0})


In [15]:
# Diabetes: my decision policy on pre-diabetes (4)
# 1 and 2 :  (Diabetes), 3: (No Diabetes), 4: (Pre-diabetes/ borderline)->treated as Yes, so encoding 1,2,4 as 1 and 3 as 0
df["Diabetes"] = df["Diabetes"].replace({1:1,2:1,3:0,4:1})

In [16]:
df = df.copy()
# checking the distribution of target variable after cleaning
df.groupby("Diabetes").size()

Diabetes
0.0    207406
1.0     41938
dtype: int64

#### Cleaned data is biased with non diabetes more than diabetes 

In [17]:
# lets see the cleaned data
df.head()

,Diabetes,HighBp,HighCholesterol,CheckedCholesterol,BMI,Smoke100,Stroke,HeartDisease,PhysicalActivity,Fruits,...,HealthCareAccess,MedicalCost,GeneralHealth,MentalHealth,PhysicalHealth,DifficultyWalking,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.2,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.1,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3,0.0,0.0,0.0,0.0,7.0,6.0,1.0
3,0.0,1.0,1.0,1.0,28.2,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,1,30.0,30.0,1.0,0.0,9.0,4.0,8.0
5,0.0,1.0,0.0,1.0,26.5,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,4,0.0,0.0,0.0,0.0,11.0,3.0,6.0
6,0.0,1.0,1.0,1.0,23.9,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,4,3.0,0.0,0.0,0.0,11.0,5.0,4.0


In [18]:
df.shape

(249344, 22)

## Saving to csv 

In [19]:
df.to_csv("../data/processed/BRFSS-2015-cleaned.csv",sep = ",",index = False)